In [1]:
import os
import multiprocessing
multiprocessing.cpu_count()

16

In [2]:
from importlib import reload
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet.contrib import amp
import time
from tqdm.notebook import tqdm

import gluoncv as gcv

In [3]:
# Dataset Loading & Transforming
image_size = 640

input_transform_fn = mx.gluon.data.vision.transforms.Compose([
    mx.gluon.data.vision.transforms.Resize(image_size, keep_ratio=True),
    mx.gluon.data.vision.transforms.CenterCrop(image_size),
    mx.gluon.data.vision.transforms.ToTensor(),
    mx.gluon.data.vision.transforms.Normalize([.485, .456, .406], [.229, .224, .225])
])

to_gpu_fn = lambda x: x.as_in_context(mx.gpu())

input_transform_fn_gpu = mx.gluon.data.vision.transforms.Compose([
    to_gpu_fn,
    input_transform_fn
])

# Pre-processing in GPU, with transforms
# then copying back to CPU memory space

# Pre-processing in GPU, with transforms
# Unfortunately, we cannot copy directly into GPU the labels
# Not supported ty ADE20KSegmentation class

to_cpu_fn = lambda x: x.as_in_context(mx.cpu())

input_transform_fn_gpu_cpu = mx.gluon.data.vision.transforms.Compose([
    input_transform_fn_gpu,
    to_cpu_fn
])

# No need for mask transform changes
ade20k_val_cpu = gcv.data.ADE20KSegmentation(split='val')

# Limit to 500 samples
max_samples = 500
samples = range(0, max_samples)

ade20k_val_cpu_pre = mx.gluon.data.SimpleDataset([(ade20k_val_cpu[i][0], ade20k_val_cpu[i][1]) for i in tqdm(samples)])
ade20k_val_gpu_cpu = ade20k_val_cpu_pre.transform_first(input_transform_fn_gpu_cpu, lazy=False)

  0%|          | 0/500 [00:00<?, ?it/s]

[21:13:20] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8600 != compiled-against version 8204.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [4]:
num_workers = 0
batch_size = 4

# DataLoader for data processed in GPU, loaded in CPU
ade20k_val_loader_gpu_cpu = mx.gluon.data.DataLoader(
    ade20k_val_gpu_cpu,
    batch_size=batch_size,
    num_workers=num_workers,
    last_batch="rollover")

## Hybridize (CPU)

In [5]:
deeplab_pt_cpu = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu.predict(data)
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/125 [00:00<?, ?it/s]

Time (s): 3.9511704444885254


In [6]:
deeplab_pt_cpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid.hybridize()

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_hybrid.predict(data)
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/125 [00:00<?, ?it/s]

Time (s): 0.6758739948272705


In [7]:
deeplab_pt_cpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid.hybridize(backend = "MKLDNN")

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_hybrid.predict(data)
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/125 [00:00<?, ?it/s]

Time (s): 0.6158411502838135


## Hybridize (GPU)

In [4]:
num_workers = 0
batch_size = 16

# DataLoader for data processed in GPU, loaded in CPU
ade20k_val_loader_gpu_cpu = mx.gluon.data.DataLoader(
    ade20k_val_cpu_pre,
    batch_size=batch_size,
    num_workers=num_workers,
    last_batch="rollover")

In [5]:
deeplab_pt_gpu = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_gpu.predict(data.as_in_context(mx.gpu()))
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/31 [00:00<?, ?it/s]

[18:27:05] /work/mxnet/src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:96: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)


Time (s): 1.6065938472747803


In [7]:
deeplab_pt_hybrid_default = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_hybrid_default.hybridize()

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_hybrid_default.predict(data.as_in_context(mx.gpu()))
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/31 [00:00<?, ?it/s]

Time (s): 0.3877408504486084


In [6]:
deeplab_pt_hybrid_static_alloc = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_hybrid_static_alloc.hybridize(static_alloc=True)

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_hybrid_static_alloc.predict(data.as_in_context(mx.gpu()))
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/31 [00:00<?, ?it/s]

Time (s): 0.29736924171447754


In [16]:
deeplab_pt_hybrid_static_alloc_shape = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_hybrid_static_alloc_shape.hybridize(static_alloc=True, static_shape=True)

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_hybrid_static_alloc_shape.predict(data.as_in_context(mx.gpu()))
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/31 [00:00<?, ?it/s]

Time (s): 0.2136681079864502


I can talk about symbolic and imperative programming
HybridSequential / HybridBlock / Hybrid_forward

https://github.com/awslabs/dynamic-training-with-apache-mxnet-on-aws/blob/master/docs/tutorials/gluon/custom_layer.md

## Automatic Mixed Precision (AMP)

In [5]:
from mxnet.contrib import amp

In [6]:
# Dataset Loading & Transforming
image_size = 1280

input_transform_fn = mx.gluon.data.vision.transforms.Compose([
    mx.gluon.data.vision.transforms.Resize(image_size, keep_ratio=True),
    mx.gluon.data.vision.transforms.CenterCrop(image_size),
    mx.gluon.data.vision.transforms.ToTensor(),
    mx.gluon.data.vision.transforms.Normalize([.485, .456, .406], [.229, .224, .225])
])

to_gpu_fn = lambda x: x.as_in_context(mx.gpu())

input_transform_fn_gpu = mx.gluon.data.vision.transforms.Compose([
    to_gpu_fn,
    input_transform_fn
])

# Pre-processing in GPU, with transforms
# then copying back to CPU memory space
to_cpu_fn = lambda x: x.as_in_context(mx.cpu())

input_transform_fn_gpu_cpu = mx.gluon.data.vision.transforms.Compose([
    input_transform_fn_gpu,
    to_cpu_fn
])

# No need for mask transform changes
ade20k_val_cpu = gcv.data.ADE20KSegmentation(split='val')

# Limit to 500 samples
max_samples = 500
samples = range(0, max_samples)

ade20k_val_cpu_pre = mx.gluon.data.SimpleDataset([(ade20k_val_cpu[i][0], ade20k_val_cpu[i][1]) for i in tqdm(samples)])
ade20k_val_gpu_cpu = ade20k_val_cpu_pre.transform_first(input_transform_fn_gpu_cpu, lazy=False)

# Single sample for forward pass (AMP requirement)
original_shape = ade20k_val_gpu_cpu[0][0].shape[1:]
single_sample_gpu = ade20k_val_gpu_cpu[0][0].reshape((1, 3) + original_shape).as_in_context(mx.gpu())

  0%|          | 0/500 [00:00<?, ?it/s]

[20:19:45] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8200 != compiled-against version 8201.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [19]:
num_workers = 0
batch_size = 16

# DataLoader for data processed in GPU, loaded in CPU
ade20k_val_loader_gpu_cpu = mx.gluon.data.DataLoader(
    ade20k_val_gpu_cpu,
    batch_size=batch_size,
    num_workers=num_workers,
    last_batch="rollover")

In [8]:
deeplab_pt_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_hybrid.predict(single_sample_gpu)
deeplab_pt_hybrid.hybridize(static_alloc=True, static_shape=True)
deeplab_pt_hybrid.forward(single_sample_gpu);

[20:19:59] /work/mxnet/src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:96: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)


In [9]:
deeplab_pt_hybrid_amp = amp.convert_hybrid_block(deeplab_pt_hybrid)

/home/ext_andres_pereztorres_oxbotica_/.local/lib/python3.7/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


In [10]:
start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_hybrid_amp.forward(data.as_in_context(mx.gpu()))
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/31 [00:00<?, ?it/s]

Time (s): 0.29921650886535645


## Quantisation (INT8)

In [4]:
from mxnet.contrib import quantization

### Calibration Dataset

In [5]:
# Dataset Loading & Transforming
# Limit to 10 samples (last ones)
max_samples = 10
samples = range(0, max_samples)

ade20k_cal_cpu_pre = mx.gluon.data.SimpleDataset([(ade20k_val_cpu[-i][0], ade20k_val_cpu[-i][1]) for i in tqdm(samples)])
ade20k_cal_gpu_cpu = ade20k_cal_cpu_pre.transform_first(input_transform_fn_gpu_cpu, lazy=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
# Datasets / DataLoaders for Calibration & Validation
# Pre-processed in GPU, copied back to CPU memory space)
num_workers = 0
batch_size = 4

# DataLoader for calibration data processed in GPU, loaded in CPU
ade20k_cal_loader_gpu_cpu = mx.gluon.data.DataLoader(
    ade20k_cal_gpu_cpu,
    batch_size=batch_size,
    num_workers=num_workers,
    last_batch="rollover")

# DataLoader for validarion data processed in GPU, loaded in CPU
ade20k_val_loader_gpu_cpu = mx.gluon.data.DataLoader(
    ade20k_val_gpu_cpu,
    batch_size=batch_size,
    num_workers=num_workers,
    last_batch="rollover")

In [7]:
# Single sample for forward pass (Quantization requirement)
original_shape = ade20k_cal_gpu_cpu[0][0].shape[1:]
single_sample_gpu = ade20k_cal_gpu_cpu[0][0].reshape((1, 3) + original_shape).as_in_context(mx.cpu())

In [8]:
deeplab_pt = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt.predict(single_sample_gpu);

In [15]:
deeplab_pt_q_hybrid = quantization.quantize_net_v2(
    deeplab_pt,
    quantized_dtype='auto',
    exclude_layers=None,
    exclude_layers_match=None,
    calib_data=ade20k_cal_loader_gpu_cpu,
    calib_mode='entropy',
    ctx=mx.cpu())

[21:15:25] /work/mxnet/src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
/home/sa_109024352321806746181/anaconda3/envs/mxnet/lib/python3.8/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


In [16]:
deeplab_pt_q_hybrid.hybridize(static_alloc=True, static_shape=True)

In [11]:
start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_hybrid_q.forward(data)
    
print("Time (s):", time.time() - start_time)

  0%|          | 0/125 [00:00<?, ?it/s]

Time (s): 0.19214582443237305


## ONNX Export

In [8]:
# ONNX library must be installed for this
# !python3 -m pip install onnx

In [9]:
deeplab_pt_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_hybrid.predict(single_sample_gpu)
deeplab_pt_hybrid.hybridize()
deeplab_pt_hybrid.forward(single_sample_gpu)

# Need to be exported externally for the symbols to be loaded
deeplab_pt_hybrid_filename = "deeplab_resnet101_coco_pt_hybrid"
deeplab_pt_hybrid.export(deeplab_pt_hybrid_filename)

# Files exported
sym_filename = deeplab_pt_hybrid_filename + "-symbol.json"
params_filename = deeplab_pt_hybrid_filename + "-0000.params"

# Verify generated files
assert os.path.exists(sym_filename)
assert os.path.exists(params_filename)

In [10]:
onnx_file_name = "deeplab_resnet101_coco_pt_hybrid.onnx"
in_shapes = [single_sample_gpu.shape]
in_types = [mx.np.float32]

onnx_model_path = mx.onnx.export_model(
    sym_filename,
    params_filename,
    in_shapes,
    in_types,
    onnx_file_name)

onnx_model_path

'deeplab_resnet101_coco_pt_hybrid.onnx'

In [11]:
# Model Verification
import onnx

# Load the ONNX model
onnx_model = onnx.load_model(onnx_model_path)

# Check the ONNX graph
onnx.checker.check_graph(onnx_model.graph)

## TensorRT Export

In [12]:
import tensorrt as trt

In [13]:
trt_file_name = "deeplab_resnet101_coco_pt_hybrid.trt"

In [14]:
TRT_LOGGER = trt.Logger(trt.Logger.INFO)
builder = trt.Builder(TRT_LOGGER)
config = builder.create_builder_config()

explicit_batch = 1 << (int) (trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
network = builder.create_network(explicit_batch)

[10/20/2023-18:30:31] [TRT] [I] [MemUsageChange] Init CUDA: CPU +297, GPU +0, now: CPU 8139, GPU 1442 (MiB)
[10/20/2023-18:30:34] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +261, GPU +74, now: CPU 8455, GPU 1516 (MiB)
[10/20/2023-18:30:34] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


In [15]:
with trt.OnnxParser(network, TRT_LOGGER) as parser:
    with open(onnx_file_name, 'rb') as model:
        parsed = parser.parse(model.read())
        engine = builder.build_engine(network, config=config)

[10/20/2023-18:30:37] [TRT] [W] onnx2trt_utils.cpp:377: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
network.num_layers 394


/tmp/ipykernel_9480/3814366024.py:7: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config=config)


[10/20/2023-18:30:39] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 8925, GPU 1524 (MiB)
[10/20/2023-18:30:39] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +10, now: CPU 8925, GPU 1534 (MiB)
[10/20/2023-18:30:39] [TRT] [I] Local timing cache in use. Profiling results in this builder pass will not be stored.
[10/20/2023-18:31:37] [TRT] [I] Total Activation Memory: 17525805568
[10/20/2023-18:31:37] [TRT] [I] Detected 1 inputs and 2 output network tensors.
[10/20/2023-18:31:37] [TRT] [I] Total Host Persistent Memory: 313440
[10/20/2023-18:31:37] [TRT] [I] Total Device Persistent Memory: 5349888
[10/20/2023-18:31:37] [TRT] [I] Total Scratch Memory: 134217728
[10/20/2023-18:31:37] [TRT] [I] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 257 MiB, GPU 8426 MiB
[10/20/2023-18:31:37] [TRT] [I] [BlockAssignment] Started assigning block shifts. This will take 125 steps to complete.
[10/20/2023-18:31:37] [TRT] [I] [BlockAssignment] Algorit

In [16]:
with open(trt_file_name, 'wb') as f:
    f.write(bytearray(engine.serialize()))

## Profiler

In [13]:
mx.profiler.set_config(
    profile_all=True,
    aggregate_stats=True,
    continuous_dump=True,
    filename='profile_output.json')

In [14]:
mx.profiler.set_state('run')

for data, _ in ade20k_val_loader_gpu_cpu:
    # deeplab_pt.forward(data)
    deeplab_pt(data)
    break

# Wait until all operations have completed
mx.nd.waitall()
# Stop recording
mx.profiler.set_state('stop')
# Log results
mx.profiler.dump()

In [19]:
mx.profiler.set_config(
    profile_all=True,
    aggregate_stats=True,
    # continuous_dump=True,
    filename='profile_output_q_hybrid.json')

In [20]:
mx.profiler.set_state('run')

for data, _ in ade20k_val_loader_gpu_cpu:
    # deeplab_pt_q_hybrid.forward(data)
    deeplab_pt_q_hybrid(data)
    break

# Wait until all operations have completed
mx.nd.waitall()
# Stop recording
mx.profiler.set_state('stop')
# Log results
mx.profiler.dump()